In [ ]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from optuna.trial import Trial
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings('ignore')

print("✅ Import thành công!")

# 1️⃣ Load dữ liệu
print("\n📊 Đang tải dữ liệu...")
data_dir = "../processed_data"  # Thay đổi đường dẫn nếu cần

try:
    X_train = pd.read_csv(f"{data_dir}/data_train.csv")
    X_val = pd.read_csv(f"{data_dir}/data_val.csv")
    X_test = pd.read_csv(f"{data_dir}/data_test.csv")
    
    # Tách features và target
    y_train = X_train.iloc[:, -1]
    X_train = X_train.iloc[:, :-1]
    
    y_val = X_val.iloc[:, -1]
    X_val = X_val.iloc[:, :-1]
    
    y_test = X_test.iloc[:, -1]
    X_test = X_test.iloc[:, :-1]
    
    # ⭐ LOẠI BỎ CÁC CỘT NON-NUMERIC
    print("🔍 Loại bỏ các cột không phải số...")
    numeric_cols = X_train.select_dtypes(include=[np.number]).columns
    X_train = X_train[numeric_cols].astype(np.float32)
    X_val = X_val[numeric_cols].astype(np.float32)
    X_test = X_test[numeric_cols].astype(np.float32)
    
    # Convert target to float
    y_train = y_train.astype(np.float32)
    y_val = y_val.astype(np.float32)
    y_test = y_test.astype(np.float32)
    
    # Xử lý NaN values
    X_train = X_train.fillna(0)
    X_val = X_val.fillna(0)
    X_test = X_test.fillna(0)
    
    y_train = y_train.fillna(0)
    y_val = y_val.fillna(0)
    y_test = y_test.fillna(0)
    
    print(f"✅ Train shape: {X_train.shape}")
    print(f"✅ Val shape: {X_val.shape}")
    print(f"✅ Test shape: {X_test.shape}")
    
except Exception as e:
    print(f"❌ Lỗi tải dữ liệu: {e}")
    print("💡 Tạo dữ liệu giả định cho demo...")
    
    # Tạo dữ liệu giả định
    from sklearn.datasets import make_regression
    X, y = make_regression(n_samples=1000, n_features=20, noise=10, random_state=42)
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    X_train = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X.shape[1])])
    X_val = pd.DataFrame(X_val, columns=[f'feature_{i}' for i in range(X.shape[1])])
    X_test = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(X.shape[1])])
    y_train = pd.Series(y_train)
    y_val = pd.Series(y_val)
    y_test = pd.Series(y_test)

c:\Users\this pc\Downloads\Final-ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Import thành công!

📊 Đang tải dữ liệu...
✅ Train shape: (66103, 31)
✅ Val shape: (14165, 31)
✅ Test shape: (14166, 31)
✅ Train shape: (66103, 31)
✅ Val shape: (14165, 31)
✅ Test shape: (14166, 31)


In [ ]:
# 2️⃣ TUNING HYPERPARAMETERS LIGHTGBM VỚI OPTUNA
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import optuna
from optuna.trial import Trial
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import joblib
import yaml
import warnings
warnings.filterwarnings('ignore')

print("\n" + "="*60)
print("🔧 BẮT ĐẦU TUNING HYPERPARAMETERS LIGHTGBM VỚI OPTUNA")
print("="*60)

# Load dữ liệu nếu chưa load
if 'X_train' not in locals():
    print("\n📊 Đang tải dữ liệu...")
    data_dir = "../processed_data"
    
    try:
        X_train = pd.read_csv(f"{data_dir}/data_train.csv")
        X_val = pd.read_csv(f"{data_dir}/data_val.csv")
        X_test = pd.read_csv(f"{data_dir}/data_test.csv")
        
        # Tách features và target
        y_train = X_train.iloc[:, -1]
        X_train = X_train.iloc[:, :-1]
        
        y_val = X_val.iloc[:, -1]
        X_val = X_val.iloc[:, :-1]
        
        y_test = X_test.iloc[:, -1]
        X_test = X_test.iloc[:, :-1]
        
        # Loại bỏ các cột non-numeric
        print("🔍 Loại bỏ các cột không phải số...")
        numeric_cols = X_train.select_dtypes(include=[np.number]).columns
        X_train = X_train[numeric_cols].astype(np.float32)
        X_val = X_val[numeric_cols].astype(np.float32)
        X_test = X_test[numeric_cols].astype(np.float32)
        
        # Convert target to float
        y_train = y_train.astype(np.float32)
        y_val = y_val.astype(np.float32)
        y_test = y_test.astype(np.float32)
        
        # Xử lý NaN values
        print(f"Train NaN trước: {X_train.isna().sum().sum()}")
        X_train = X_train.fillna(0)
        X_val = X_val.fillna(0)
        X_test = X_test.fillna(0)
        
        y_train = y_train.fillna(0)
        y_val = y_val.fillna(0)
        y_test = y_test.fillna(0)
        print(f"Train NaN sau: {X_train.isna().sum().sum()}")
        
        print(f"✅ Train shape: {X_train.shape}")
        print(f"✅ Val shape: {X_val.shape}")
        print(f"✅ Test shape: {X_test.shape}")
        
    except Exception as e:
        print(f"❌ Lỗi tải dữ liệu: {e}")
        print("💡 Tạo dữ liệu giả định cho demo...")
        
        # Tạo dữ liệu giả định
        from sklearn.datasets import make_regression
        X, y = make_regression(n_samples=1000, n_features=20, noise=10, random_state=42)
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
        
        X_train = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(X.shape[1])]).astype(np.float32)
        X_val = pd.DataFrame(X_val, columns=[f'feature_{i}' for i in range(X.shape[1])]).astype(np.float32)
        X_test = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(X.shape[1])]).astype(np.float32)
        y_train = pd.Series(y_train).astype(np.float32)
        y_val = pd.Series(y_val).astype(np.float32)
        y_test = pd.Series(y_test).astype(np.float32)

def objective(trial: Trial):
    """
    Hàm objective để Optuna tối ưu hóa
    """
    # Định nghĩa không gian hyperparameter
    params = {
        'objective': 'regression',
        'metric': 'mse',
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.1),
        'random_state': 42,
        'verbose': -1,
    }
    
    # Tạo LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    
    # Train model với early stopping
    model = lgb.train(
        params,
        train_data,
        num_boost_round=100,
        valid_sets=[lgb.Dataset(X_val, label=y_val)],
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=0)  # Tắt log
        ]
    )
    
    # Đánh giá trên validation set
    y_pred = model.predict(X_val)
    mse = mean_squared_error(y_val, y_pred)
    
    return mse

# Tạo study với Optuna
print("\n📊 Bắt đầu tuning với 50 trials...")
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

# Chạy optimization
study.optimize(objective, n_trials=50, show_progress_bar=True)

# 3️⃣ Hiển thị kết quả
print("\n" + "="*60)
print("✅ HOÀN THÀNH TUNING!")
print("="*60)

# Best trial
best_trial = study.best_trial
print(f"\n🏆 Best Trial: {best_trial.number}")
print(f"📊 Best MSE: {best_trial.value:.6f}")
print(f"\n📝 Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

# 4️⃣ Train mô hình final với best hyperparameters
print("\n" + "="*60)
print("🚀 TRAIN MÔ HÌNH FINAL VỚI BEST HYPERPARAMETERS")
print("="*60)

best_params = study.best_trial.params
best_params['objective'] = 'regression'
best_params['metric'] = 'mse'

train_data = lgb.Dataset(X_train, label=y_train)

final_model = lgb.train(
    best_params,
    train_data,
    num_boost_round=200,
    valid_sets=[lgb.Dataset(X_val, label=y_val)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=20),
        lgb.log_evaluation(period=20)
    ]
)

# 5️⃣ Đánh giá mô hình
print("\n" + "="*60)
print("📊 ĐÁNH GIÁ MÔ HÌNH")
print("="*60)

# Train evaluation
y_train_pred = final_model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print(f"\n🏃 Train Metrics:")
print(f"  MSE:  {train_mse:.6f}")
print(f"  MAE:  {train_mae:.6f}")
print(f"  R²:   {train_r2:.6f}")

# Val evaluation
y_val_pred = final_model.predict(X_val)
val_mse = mean_squared_error(y_val, y_val_pred)
val_mae = mean_absolute_error(y_val, y_val_pred)
val_r2 = r2_score(y_val, y_val_pred)

print(f"\n✔️ Val Metrics:")
print(f"  MSE:  {val_mse:.6f}")
print(f"  MAE:  {val_mae:.6f}")
print(f"  R²:   {val_r2:.6f}")

# Test evaluation
y_test_pred = final_model.predict(X_test)
test_mse = mean_squared_error(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"\n🎯 Test Metrics:")
print(f"  MSE:  {test_mse:.6f}")
print(f"  MAE:  {test_mae:.6f}")
print(f"  R²:   {test_r2:.6f}")

# 6️⃣ Lưu mô hình
print("\n" + "="*60)
print("💾 LƯU MÔ HÌNH")
print("="*60)

model_path = "lightgbm_best_model.pkl"
joblib.dump(final_model, model_path)
print(f"✅ Mô hình đã được lưu: {model_path}")

# Lưu best hyperparameters
params_path = "lightgbm_best_params.yaml"
with open(params_path, 'w') as f:
    yaml.dump(best_params, f, default_flow_style=False)
print(f"✅ Best hyperparameters đã được lưu: {params_path}")

print("\n✅ Quá trình tuning hoàn thành!")

[I 2025-11-16 20:59:18,365] A new study created in memory with name: no-name-d0465f36-11a0-45ab-995b-e2b0ba2b37e9



🔧 BẮT ĐẦU TUNING HYPERPARAMETERS LIGHTGBM VỚI OPTUNA

📊 Bắt đầu tuning với 50 trials...


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

[W 2025-11-16 20:59:18,371] Trial 0 failed with parameters: {'num_leaves': 69, 'max_depth': 15, 'learning_rate': 0.1205712628744377, 'min_child_samples': 32, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'reg_alpha': 0.05808361216819946, 'reg_lambda': 0.8661761457749352, 'min_split_gain': 0.06011150117432088} because of the following error: ValueError('pandas dtypes must be int, float or bool.\nFields with bad pandas dtypes: datetime: object, name: object, datetime.1: object, preciptype: object, conditions: object, icon: object, stations: object').
Traceback (most recent call last):
  File "c:\Users\this pc\Downloads\Final-ML\.venv\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\this pc\AppData\Local\Temp\ipykernel_6636\3985308511.py", line 103, in objective
    model = lgb.train(
        params,
    ...<6 lines>...
        ]
    )
  File "c:\Users\this pc\Downloads\Final-ML\.venv\Lib\si

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: datetime: object, name: object, datetime.1: object, preciptype: object, conditions: object, icon: object, stations: object